In [1]:
# import libraries
import os, sys
import string
import pandas as pd
try:
    import nltk
except:
    !conda install --yes --prefix {sys.prefix} nltk
    import nltk

# import modules
from utils.preprocessing import get_dataframe, label_encoding, feature_engineering

In [2]:
# from sklearn.tree import DecisionTreeClassifier

In [3]:
forced = [
#     "comma_per_word",
#     "point_per_word",
#     "ellipsis_per_word",
    ]

### 1) File Exploration

In [4]:
# ver conteúdos da pasta
os.listdir()

['.git',
 '.gitignore',
 '.ipynb_checkpoints',
 'BOA_project_description.pdf',
 'corpora',
 'corpora_preprocessing.ipynb',
 'LICENSE',
 'README.md',
 'requirements.txt',
 'utils']

In [5]:
authors = os.listdir("corpora/train")

In [6]:
for author in authors:
    for excerto in [x for x in os.listdir(f"corpora/train/{author}") if x.endswith(".txt")]:
        print(f"corpora/train/{author}/{excerto}")

corpora/train/AlmadaNegreiros/pg22615.txt
corpora/train/AlmadaNegreiros/pg22730.txt
corpora/train/AlmadaNegreiros/pg22801.txt
corpora/train/AlmadaNegreiros/pg22802.txt
corpora/train/AlmadaNegreiros/pg22969.txt
corpora/train/AlmadaNegreiros/pg23133.txt
corpora/train/AlmadaNegreiros/pg23620.txt
corpora/train/AlmadaNegreiros/pg23879.txt
corpora/train/AlmadaNegreiros/pg23961.txt
corpora/train/CamiloCasteloBranco/24691-0.txt
corpora/train/CamiloCasteloBranco/34756-0.txt
corpora/train/CamiloCasteloBranco/pg16425.txt
corpora/train/CamiloCasteloBranco/pg17927.txt
corpora/train/CamiloCasteloBranco/pg19375.txt
corpora/train/CamiloCasteloBranco/pg21406.txt
corpora/train/CamiloCasteloBranco/pg23203.txt
corpora/train/CamiloCasteloBranco/pg23345.txt
corpora/train/CamiloCasteloBranco/pg23346.txt
corpora/train/CamiloCasteloBranco/pg24339.txt
corpora/train/CamiloCasteloBranco/pg25844.txt
corpora/train/CamiloCasteloBranco/pg26017.txt
corpora/train/CamiloCasteloBranco/pg26103.txt
corpora/train/CamiloCast

___
### 2) Train

In [7]:
train = get_dataframe(path_to_train="corpora/train/", author_list=authors,
                      preserve_blank_lines=False, join_every_line=True,
                      separator=" ")

In [8]:
train.head()

,text,author
0,Title: A Scena do Odio Author: José de Almada ...,AlmadaNegreiros
1,Title: O Jardim da Pierrette Author: José de A...,AlmadaNegreiros
2,Title: A Invenção do Dia Claro Author: José de...,AlmadaNegreiros
3,Title: Litoral A Amadeo de Souza Cardozo Autho...,AlmadaNegreiros
4,EXPOSIÇÃO +amadeo de souza cardoso+ LIGA NAVAL...,AlmadaNegreiros


In [9]:
# add new features
train = feature_engineering(train, "text")

train.head()

,text,author,ellipsis_per_word,punct_count,:,",",.,-,!,?,;
0,Title: A Scena do Odio Author: José de Almada ...,AlmadaNegreiros,0.003939,473,0.003939,0.066404,0.016882,0.077659,0.099606,0.001688,0.000000
1,Title: O Jardim da Pierrette Author: José de A...,AlmadaNegreiros,0.000000,42,0.015723,0.022013,0.059748,0.031447,0.003145,0.000000,0.000000
2,Title: A Invenção do Dia Claro Author: José de...,AlmadaNegreiros,0.001656,1092,0.005962,0.043723,0.059954,0.035939,0.029977,0.002319,0.002981
3,Title: Litoral A Amadeo de Souza Cardozo Autho...,AlmadaNegreiros,0.000000,37,0.007634,0.001527,0.001527,0.038168,0.006107,0.001527,0.000000
4,EXPOSIÇÃO +amadeo de souza cardoso+ LIGA NAVAL...,AlmadaNegreiros,0.000000,88,0.001727,0.050086,0.056995,0.032815,0.003454,0.003454,0.003454


In [10]:
# codificação da coluna y para fornecer ao futuro modelo
train, le = label_encoding(train, "author")

In [11]:
# train = clean_text(train, "text")
# train.head()

In [12]:
import numpy as np

In [13]:
# import string
# from collections import Counter

# count = lambda l1,l2: sum([1 for x in l1 if x in l2])

# selected_punct = "!,.-:;?"
# # train.text.apply(lambda s: {k:v for k, v in Counter(s).items() if k in string.punctuation}).apply(pd.Series).fillna(0).astype("int")
# train.text.apply(lambda s: {k:v for k, v in Counter(s).items() if k in selected_punct}).apply(pd.Series).fillna(0).astype("int").divide(np.random.randint(1, 9999, 63), axis=0)

# # train.text.apply(lambda s: count(s, selected_punct))

In [14]:
# # TODO cols:  
# train["punctuation"]

In [15]:
# reorder columns
Xcols = list(set(train.columns) - set(["author"]) - set(["text"]))
Xcols.sort()

cols = ["text"]
cols.extend(Xcols)
cols.append("author")

train = train[cols]
train.head()

,text,!,",",-,.,:,;,?,ellipsis_per_word,punct_count,author
0,Title: A Scena do Odio Author: José de Almada ...,0.099606,0.066404,0.077659,0.016882,0.003939,0.000000,0.001688,0.003939,473,0
1,Title: O Jardim da Pierrette Author: José de A...,0.003145,0.022013,0.031447,0.059748,0.015723,0.000000,0.000000,0.000000,42,0
2,Title: A Invenção do Dia Claro Author: José de...,0.029977,0.043723,0.035939,0.059954,0.005962,0.002981,0.002319,0.001656,1092,0
3,Title: Litoral A Amadeo de Souza Cardozo Autho...,0.006107,0.001527,0.038168,0.001527,0.007634,0.000000,0.001527,0.000000,37,0
4,EXPOSIÇÃO +amadeo de souza cardoso+ LIGA NAVAL...,0.003454,0.050086,0.032815,0.056995,0.001727,0.003454,0.003454,0.000000,88,0


In [16]:
if len(forced) > 0:
    Xcols = forced

X = train[Xcols]
y = train["author"]

___
### 3) Test

In [17]:
test = get_dataframe("corpora/test/", author_list=["1000Palavras", "500Palavras"],
                     preserve_blank_lines=False, join_every_line=True,
                     separator=" ")

test["author"] = test["author"].str.replace("1000Palavras", "1000").str.replace("500Palavras", "500").astype(int)
test.columns = [list(test.columns)[0], "word_count"]
test.head()

,text,word_count
0,"Depois, pouco a pouco, a tranquilidade regress...",1000
1,Justamente como se eu tivesse tido a ideia de ...,1000
2,"Quase um mês depois, a época de exames aproxim...",1000
3,"Agora, porém, era sem fervor, arrastadamente, ...",1000
4,"O cahos de cima a descer, a descer com a morta...",1000


In [18]:
# add new features
test = feature_engineering(test, "text")
test.drop("word_count", axis=1, inplace=True)
test[Xcols].head()

,!,",",-,.,:,;,?,ellipsis_per_word,punct_count
0,0.000000,0.169028,0.025304,0.017206,0.000000,0.000000,0.000000,0.000000,209
1,0.002134,0.028815,0.030950,0.036286,0.001067,0.001067,0.006403,0.000000,100
2,0.001908,0.047710,0.025763,0.085878,0.003817,0.001908,0.009542,0.014313,185
3,0.012085,0.144008,0.035247,0.040282,0.005035,0.009063,0.003021,0.005035,247
4,0.000000,0.114201,0.023833,0.038729,0.003972,0.006951,0.000993,0.000000,190


___

### 4) Model predictions

In [19]:
import numpy as np
from sklearn.naive_bayes import GaussianNB

clf = GaussianNB()
clf.fit(X, y)

GaussianNB(priors=None, var_smoothing=1e-09)

In [20]:
test["author_pred"] = clf.predict(test[Xcols])
test["author_pred"] = le.inverse_transform(test["author_pred"])

In [21]:
test

,text,ellipsis_per_word,punct_count,",",-,.,?,!,;,:,author_pred
0,"Depois, pouco a pouco, a tranquilidade regress...",0.000000,209,0.169028,0.025304,0.017206,0.000000,0.000000,0.000000,0.000000,LuisaMarquesSilva
1,Justamente como se eu tivesse tido a ideia de ...,0.000000,100,0.028815,0.030950,0.036286,0.006403,0.002134,0.001067,0.001067,LuisaMarquesSilva
2,"Quase um mês depois, a época de exames aproxim...",0.014313,185,0.047710,0.025763,0.085878,0.009542,0.001908,0.001908,0.003817,LuisaMarquesSilva
3,"Agora, porém, era sem fervor, arrastadamente, ...",0.005035,247,0.144008,0.035247,0.040282,0.003021,0.012085,0.009063,0.005035,LuisaMarquesSilva
4,"O cahos de cima a descer, a descer com a morta...",0.000000,190,0.114201,0.023833,0.038729,0.000993,0.000000,0.006951,0.003972,LuisaMarquesSilva
5,"""O Senhor ensina pela pena o que o homem não s...",0.002035,155,0.066124,0.017294,0.059003,0.010173,0.001017,0.002035,0.002035,LuisaMarquesSilva
6,"Depois, pouco a pouco, a tranquilidade regress...",0.000000,94,0.159274,0.014113,0.016129,0.000000,0.000000,0.000000,0.000000,LuisaMarquesSilva
7,Justamente como se eu tivesse tido a ideia de ...,0.000000,54,0.041408,0.024845,0.028986,0.012422,0.002070,0.002070,0.000000,LuisaMarquesSilva
8,"Quase um mês depois, a época de exames aproxim...",0.013807,91,0.041420,0.037475,0.078895,0.011834,0.003945,0.001972,0.003945,LuisaMarquesSilva
9,"Agora, porém, era sem fervor, arrastadamente, ...",0.004090,113,0.139059,0.034765,0.028630,0.000000,0.004090,0.018405,0.006135,LuisaMarquesSilva


___
### 5) Hard evaluation

Evaluating on test set given the fact that the train dataset is small for train/valid split, which is not desirable.

In [22]:
# labels feitas à mão
labels = ["JoseSaramago", "AlmadaNegreiros", "LuisaMarquesSilva",
          "EcaDeQueiros", "CamiloCasteloBranco", "JoseRodriguesSantos",
          "JoseSaramago", "LuisaMarquesSilva", "LuisaMarquesSilva",
          "EcaDeQueiros", "CamiloCasteloBranco", "JoseRodriguesSantos"]

In [23]:
score, certos = 0, []

for index, i in enumerate(test["author_pred"]):
    if i == labels[index]:
        print(f"#{index}:\tcerto ({i})")
        score += 1
        certos.append(index)
    else:
        print(f"#{index}:\terrado")

accuracy = round(score/len(test)*100, 2)
print(f"\naccuracy = {accuracy} %")

#0:	errado
#1:	errado
#2:	certo (LuisaMarquesSilva)
#3:	errado
#4:	errado
#5:	errado
#6:	errado
#7:	certo (LuisaMarquesSilva)
#8:	certo (LuisaMarquesSilva)
#9:	errado
#10:	errado
#11:	errado

accuracy = 25.0 %


In [24]:
print(f"\tforced: \t{forced}\n\tcertos: \t{', '.join([(f'{n}') for n in certos])}\n\taccuracy:\t{accuracy} %")

	forced: 	[]
	certos: 	2, 7, 8
	accuracy:	25.0 %


In [25]:
print(test.columns.tolist())

['text', 'ellipsis_per_word', 'punct_count', ',', '-', '.', '?', '!', ';', ':', 'author_pred']


___
### **[TODO] features:**
- <s>número de caracteres por linha</s>
- <s>número de palavras por linha</s>
- <s>número de vírgulas por frase</s>
- <s>vírgulas utilizadas por palavra</s>
- <s>número de ellipsis por palavra</s>
- número de caracteres especiais (pontuação) por palavra